# **Read & Imports**

In [1]:
##imports

# Suppress warnings
import warnings
import logging
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from fastai.tabular.all import *
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import PolynomialFeatures, KBinsDiscretizer
from sklearn.preprocessing import FunctionTransformer
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch.nn.init as init
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
import joblib
from sklearn.ensemble import VotingRegressor
from sklearn.pipeline import Pipeline
import optuna
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, LSTM, GRU, Input
from keras.utils import to_categorical


In [2]:
## read data

df = pd.read_csv('/kaggle/input/la-data-full/LosAngeles_Earthquake_Dataset.csv')
df['class'].value_counts()

class
2    6343
3    5201
1    4711
4    3437
5    2633
6     574
Name: count, dtype: int64

# **Check Data Leakage**

In [3]:
## Check any leakage (use full dataset)
df = pd.read_csv('/kaggle/input/la-data-full/LosAngeles_Earthquake_Dataset.csv')

X = df.drop('class', axis=1)  
y = df['class']

# Split data into training and testing sets (80% train, 20% test), random split with leakage :D
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize Random Forest model with specified parameters in the paper
rf_model = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='sqrt',
    random_state=15
)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)
y_pred_prob = rf_model.predict_proba(X_test)


In [4]:
# Evaluate
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
y_test_bin = label_binarize(y_test, classes=np.unique(y_train))  # Convert y_test to a binary array format for ROC-AUC
roc_auc = roc_auc_score(y_test_bin, y_pred_prob, average='weighted', multi_class='ovr')

print("Model Evaluation Metrics:")
print(f"Model Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

Model Evaluation Metrics:
Model Accuracy: 0.9769
Precision: 0.9769
Recall: 0.9769
F1-Score: 0.9768
ROC-AUC: 0.9989


**We have the exact same results with the paper. I also calculated these metrics by hand, it's the same result with libraries.**

## **All Models With Data Leakage**

In [5]:
# Function to evaluate model
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test) if hasattr(model, "predict_proba") else None ## For ROC-AUC

    accuracy = round(accuracy_score(y_test, y_pred), 4)
    precision = round(precision_score(y_test, y_pred, average='weighted', zero_division=0), 4)
    recall = round(recall_score(y_test, y_pred, average='weighted'), 4)
    f1 = round(f1_score(y_test, y_pred, average='weighted'), 4)

    ##ROC-AUC
    if y_pred_prob is not None:
        y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
        roc_auc = round(roc_auc_score(y_test_bin, y_pred_prob, average='weighted', multi_class='ovr'), 4)
    else:
        roc_auc = None

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "ROC-AUC": roc_auc
    }

# Function to evaluate neural network models
def evaluate_nn_model(model, X_train, X_test, y_train, y_test):
    # Convert labels to categorical
    y_train_cat = to_categorical(y_train, num_classes=len(np.unique(y_train)))
    y_test_cat = to_categorical(y_test, num_classes=len(np.unique(y_train)))

    # Compile and train the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train_cat, epochs=10, batch_size=32, verbose=0)

    # Predict probabilities and classes
    y_pred_prob = model.predict(X_test, verbose=0)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # Calculate metrics
    accuracy = round(accuracy_score(y_test, y_pred), 4)
    precision = round(precision_score(y_test, y_pred, average='weighted', zero_division=0), 4)
    recall = round(recall_score(y_test, y_pred, average='weighted'), 4)
    f1 = round(f1_score(y_test, y_pred, average='weighted'), 4)

    # ROC-AUC calculation using y_test_cat
    roc_auc = round(roc_auc_score(y_test_cat, y_pred_prob, average='weighted', multi_class='ovr'), 4)

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "ROC-AUC": roc_auc
    }


In [6]:
# Load dataset
df = pd.read_csv('/kaggle/input/la-data-full/LosAngeles_Earthquake_Dataset.csv')
X = df.drop('class', axis=1)
y = df['class']

# Map class labels to a consistent range (e.g., [0, 1, 2, ...])
class_mapping = {original: new for new, original in enumerate(sorted(y.unique()))}
y_mapped = y.map(class_mapping)

# Split dataset using the mapped labels
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.2, random_state=15)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Models to evaluate
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, criterion='gini', max_features='sqrt', random_state=15),
    "Logistic Regression": LogisticRegression(solver='lbfgs', max_iter=1000, random_state=15),
    "Decision Trees": DecisionTreeClassifier(criterion='gini', random_state=15),
    "Gradient Boosting Machines": GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, random_state=15),
    "Support Vector Machines": SVC(kernel='rbf', C=1.0, gamma='scale', random_state=15, probability=True),
    "k-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='minkowski'),
    "Naive Bayes": GaussianNB(),
    "AdaBoost": AdaBoostClassifier(n_estimators=50, algorithm='SAMME', random_state=15),
    "XGBoost": XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=15),
    "LightGBM": LGBMClassifier(num_leaves=64, max_depth=6, min_split_gain=0.5, min_child_samples=20, random_state=15, verbose=-1),
    "Multilayer Perceptron": MLPClassifier(hidden_layer_sizes=(100, 100), learning_rate_init=0.001, max_iter=1000, random_state=15)
}

# Neural network models to evaluate
nn_models = {
    "Convolutional Neural Network": Sequential([
        Input(shape=(X_train.shape[1], 1)),
        Conv1D(32, kernel_size=3, activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')
    ]),
    "Recurrent Neural Network (RNN)": Sequential([
        Input(shape=(X_train.shape[1], 1)),
        LSTM(64, return_sequences=True),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')
    ]),
    "Long Short-Term Memory (LSTM)": Sequential([
        Input(shape=(X_train.shape[1], 1)),
        LSTM(64),
        Dense(64, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')
    ]),
    "Gated Recurrent Unit (GRU)": Sequential([
        Input(shape=(X_train.shape[1], 1)),
        GRU(64),
        Dense(64, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')
    ]),
    "Transformer Model": Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')
    ])
}

# Evaluate all models
results = []
for name, model in models.items():
    print(f"Evaluating {name}...")
    metrics = evaluate_model(model, X_train, X_test, y_train, y_test)
    results.append({"Model": name, **metrics})

# Reshape X_train and X_test for neural network models
X_train_nn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_nn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

for name, model in nn_models.items():
    print(f"Evaluating {name}...")
    metrics = evaluate_nn_model(model, X_train_nn, X_test_nn, y_train, y_test)
    results.append({"Model": name, **metrics})

# Create a DataFrame to summarize results
results_df = pd.DataFrame(results)

# Define custom order of models for display
model_order = [
    "Logistic Regression", "Decision Trees", "Random Forest", "Gradient Boosting Machines", 
    "Support Vector Machines", "k-Nearest Neighbors", "Naive Bayes", "AdaBoost", 
    "XGBoost", "LightGBM", "Multilayer Perceptron", "Convolutional Neural Network", 
    "Recurrent Neural Network (RNN)", "Long Short-Term Memory (LSTM)", 
    "Gated Recurrent Unit (GRU)", "Transformer Model"
]

# Reorder the DataFrame
results_df["Model"] = pd.Categorical(results_df["Model"], categories=model_order, ordered=True)
results_df.sort_values(by="Model", inplace=True)
results_df.reset_index(drop=True, inplace=True)

Evaluating Random Forest...
Evaluating Logistic Regression...
Evaluating Decision Trees...
Evaluating Gradient Boosting Machines...
Evaluating Support Vector Machines...
Evaluating k-Nearest Neighbors...
Evaluating Naive Bayes...
Evaluating AdaBoost...
Evaluating XGBoost...
Evaluating LightGBM...
Evaluating Multilayer Perceptron...
Evaluating Convolutional Neural Network...
Evaluating Recurrent Neural Network (RNN)...
Evaluating Long Short-Term Memory (LSTM)...
Evaluating Gated Recurrent Unit (GRU)...
Evaluating Transformer Model...


In [7]:
results_df

,Model,Accuracy,Precision,Recall,F1-Score,ROC-AUC
0,Logistic Regression,0.3262,0.3447,0.3262,0.2999,0.6255
1,Decision Trees,0.8967,0.8969,0.8967,0.8967,0.9348
2,Random Forest,0.9769,0.9769,0.9769,0.9768,0.9989
3,Gradient Boosting Machines,0.7007,0.7163,0.7007,0.7010,0.9197
4,Support Vector Machines,0.5207,0.5835,0.5207,0.5071,0.8118
5,k-Nearest Neighbors,0.6384,0.6483,0.6384,0.6372,0.8697
6,Naive Bayes,0.2620,0.2954,0.2620,0.2420,0.5742
7,AdaBoost,0.3533,0.4418,0.3533,0.3229,0.6481
8,XGBoost,0.9007,0.9025,0.9007,0.9008,0.9869
9,LightGBM,0.9336,0.9341,0.9336,0.9336,0.9936


# **Same Models Without Data Leakage**

In [8]:
# Load dataset
df = pd.read_csv('/kaggle/input/la-data-full/LosAngeles_Earthquake_Dataset.csv')
X = df.drop('class', axis=1)
y = df['class']

# Map class labels to a consistent range (e.g., [0, 1, 2, ...])
class_mapping = {original: new for new, original in enumerate(sorted(y.unique()))}
y_mapped = y.map(class_mapping)

# Split dataset into training and validation sets (80% train, 20% validation) ## the right way.
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y_mapped.iloc[:split_index], y_mapped.iloc[split_index:]

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Models to evaluate
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, criterion='gini', max_features='sqrt', random_state=15),
    "Logistic Regression": LogisticRegression(solver='lbfgs', max_iter=1000, random_state=15),
    "Decision Trees": DecisionTreeClassifier(criterion='gini', random_state=15),
    "Gradient Boosting Machines": GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, random_state=15),
    "Support Vector Machines": SVC(kernel='rbf', C=1.0, gamma='scale', random_state=15, probability=True),
    "k-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='minkowski'),
    "Naive Bayes": GaussianNB(),
    "AdaBoost": AdaBoostClassifier(n_estimators=50, algorithm='SAMME', random_state=15),
    "XGBoost": XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=15),
    "LightGBM": LGBMClassifier(num_leaves=64, max_depth=6, min_split_gain=0.5, min_child_samples=20, random_state=15, verbose=-1),
    "Multilayer Perceptron": MLPClassifier(hidden_layer_sizes=(100, 100), learning_rate_init=0.001, max_iter=1000, random_state=15)
}

# Neural network models to evaluate
nn_models = {
    "Convolutional Neural Network": Sequential([
        Input(shape=(X_train.shape[1], 1)),
        Conv1D(32, kernel_size=3, activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')
    ]),
    "Recurrent Neural Network (RNN)": Sequential([
        Input(shape=(X_train.shape[1], 1)),
        LSTM(64, return_sequences=True),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')
    ]),
    "Long Short-Term Memory (LSTM)": Sequential([
        Input(shape=(X_train.shape[1], 1)),
        LSTM(64),
        Dense(64, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')
    ]),
    "Gated Recurrent Unit (GRU)": Sequential([
        Input(shape=(X_train.shape[1], 1)),
        GRU(64),
        Dense(64, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')
    ]),
    "Transformer Model": Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')
    ])
}

# Evaluate all models
results = []
for name, model in models.items():
    print(f"Evaluating {name}...")
    metrics = evaluate_model(model, X_train, X_test, y_train, y_test)
    results.append({"Model": name, **metrics})

# Reshape X_train and X_test for neural network models
X_train_nn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_nn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

for name, model in nn_models.items():
    print(f"Evaluating {name}...")
    metrics = evaluate_nn_model(model, X_train_nn, X_test_nn, y_train, y_test)
    results.append({"Model": name, **metrics})

# Create a DataFrame to summarize results
results_df = pd.DataFrame(results)

# Define custom order of models for display
model_order = [
    "Logistic Regression", "Decision Trees", "Random Forest", "Gradient Boosting Machines", 
    "Support Vector Machines", "k-Nearest Neighbors", "Naive Bayes", "AdaBoost", 
    "XGBoost", "LightGBM", "Multilayer Perceptron", "Convolutional Neural Network", 
    "Recurrent Neural Network (RNN)", "Long Short-Term Memory (LSTM)", 
    "Gated Recurrent Unit (GRU)", "Transformer Model"
]

# Reorder the DataFrame
results_df["Model"] = pd.Categorical(results_df["Model"], categories=model_order, ordered=True)
results_df.sort_values(by="Model", inplace=True)
results_df.reset_index(drop=True, inplace=True)

Evaluating Random Forest...
Evaluating Logistic Regression...
Evaluating Decision Trees...
Evaluating Gradient Boosting Machines...
Evaluating Support Vector Machines...
Evaluating k-Nearest Neighbors...
Evaluating Naive Bayes...
Evaluating AdaBoost...
Evaluating XGBoost...
Evaluating LightGBM...
Evaluating Multilayer Perceptron...
Evaluating Convolutional Neural Network...
Evaluating Recurrent Neural Network (RNN)...
Evaluating Long Short-Term Memory (LSTM)...
Evaluating Gated Recurrent Unit (GRU)...
Evaluating Transformer Model...


In [9]:
results_df

,Model,Accuracy,Precision,Recall,F1-Score,ROC-AUC
0,Logistic Regression,0.2216,0.2777,0.2216,0.1750,0.5590
1,Decision Trees,0.2083,0.1995,0.2083,0.1925,0.5127
2,Random Forest,0.2129,0.1842,0.2129,0.1718,0.5086
3,Gradient Boosting Machines,0.1993,0.1950,0.1993,0.1794,0.4888
4,Support Vector Machines,0.1897,0.1881,0.1897,0.1432,0.4752
5,k-Nearest Neighbors,0.2050,0.1964,0.2050,0.1874,0.5049
6,Naive Bayes,0.2295,0.2357,0.2295,0.1974,0.5423
7,AdaBoost,0.3114,0.2468,0.3114,0.2523,0.5929
8,XGBoost,0.2083,0.1911,0.2083,0.1809,0.4945
9,LightGBM,0.2140,0.1962,0.2140,0.1848,0.5000
